In [1]:
import pandas as pd
import numpy as np
import sys, os , time, pickle
from tabulate import tabulate
from IPython.display import display

def find_match(string,list_des):
    return [value for value in list_des if string in value]

def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(file_name)
    display(table)
    #print(tabulate(table, headers='keys', tablefmt='rst')) #latex
    
def calculate_aux(archivo,what_to_do="mv",model=""):
    try:
        f = open(archivo)
        result_list = []
        for line in f.readlines():
            if what_to_do == "mv":
                if "ACC MV" in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
            elif what_to_do == 'epochs':
                if "Epochs to converge" in line and model in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
        f.close()
        return result_list
    except:
        return "-"
    
def compare_table(tipo,repre="normal", std=False):
    t = pd.DataFrame()
    t["Methods"] = ["Ann(tr)"]+methods_list
    t["Train"] = ["-"] + ["-" for _ in range(len(methods_list))]
    t["Test"] = ["-"] + ["-" for _ in range(len(methods_list))]
    if tipo == 'acc':
        try:
            t.loc[0,"Train"] = np.mean(calculate_aux(folder+"results.txt",what_to_do="mv"))
        except:
            try:
                t.loc[0,"Train"] = np.mean(calculate_aux(folder+"results_ind.txt",what_to_do="mv"))
            except:
                print("No se encontró acc")
            
    for k, method in enumerate(methods_list):
        aux = find_match(method,files)
        if tipo == "acc": #como normal?
            if std:
                test_aux = find_match("_train_std.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])["Accuracy"][0], 4)
                train_aux = find_match("_test_std.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])["Accuracy"][0], 4)
            else:
                test_aux = find_match("_train.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])["Accuracy"][0], 4)
                train_aux = find_match("_test.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])["Accuracy"][0], 4)

        elif "norm" in tipo.lower() or "js" in tipo.lower():
            if std:
                kl_aux = find_match("_train_std.csv",aux)
            else:
                kl_aux = find_match("_train.csv",aux)
            
            if len(kl_aux) != 0:                
                if repre=="R":
                    to_extract = "("+repre+") "+tipo+ " mean"
                elif repre=='G':
                    to_extract = "("+repre+") "+tipo
                try:
                    t.loc[k+1,"Train"] = round(pd.read_csv(folder+kl_aux[-1])[to_extract][0] ,4)
                except:
                    pass
        elif tipo == "epochs":
            methods_list_aux = ["soft-MV","hard-MV", "D&S", "Raykar", "Ours Global","Ours Individual","2", "3"]
            t.loc[k+1,"Train"]=calculate_aux(folder+"results.txt",what_to_do="epochs",model=methods_list_aux[k])
    return t

def get_value(lista, name, std=False):
    i = 0
    if std:
        i = 1
    return np.asarray([value[i][name][0] if type(value[i])== pd.DataFrame else -9999 for value in lista] )

def create_df(method_l,T_try, value, file_re = "./results.txt"):
    t = pd.DataFrame(np.zeros((len(method_l)+1,len(T_try))))    
    t.columns = T_try
    t.index = ["Ann(tr)"]+method_l
    t_std = t.copy()
    
    if value.lower() == "accuracy" or "F1" in value.lower():
        if len(calculate_aux(file_re,what_to_do="mv")) > len(T_try) +1:
            aux2 = calculate_aux(file_re,what_to_do="mv")
            aux = [np.mean(aux2[i:30*(i+1)]) for i in range(0,len(aux2),30)]
        else:
            aux = calculate_aux(file_re,what_to_do="mv")
        t.loc["Ann(tr)",:] = aux
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["test"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["test"], value, std=True)

    elif "inertia" in value.lower() or "iner" in value.lower():
        for method_l in methods_list:
            if "Ours" in method_l:
                t.loc[method_l,:] = get_value(dic_res[method_l]["testAux"], value)
                
    else:
        t.index = ["D_t"]+method_l
        t.loc["D_t",:] = D_t
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["train"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["train"], value, std=True)
    return t.round(4), t_std.round(4)

!ls

CIFAR_T  I_fix	LabelMe  LabelMe_v2  Plot_Results.ipynb  simple_T


In [21]:
!ls I_fix/LabelMe
folder = "I_fix/LabelMe/"

files = sorted(os.listdir(folder))
methods_list = list(set([f.split("_")[1] for f in os.listdir(folder) if (len(f.split("_")) >2) and ("UpperBound" not in f)]))
methods_list

LabelMe_hardMV_test.csv
LabelMe_hardMV_test_std.csv
LabelMe_hardMV_train.csv
LabelMe_hardMV_train_std.csv
LabelMe_OursIndividual2_testAux.csv
LabelMe_OursIndividual2_testAux_std.csv
LabelMe_OursIndividual2_test.csv
LabelMe_OursIndividual2_test_std.csv
LabelMe_OursIndividual2_trainAnn.csv
LabelMe_OursIndividual2_trainAnn_std.csv
LabelMe_OursIndividual2_train.csv
LabelMe_OursIndividual2_train_std.csv
LabelMe_OursIndividual3_testAux.csv
LabelMe_OursIndividual3_testAux_std.csv
LabelMe_OursIndividual3_test.csv
LabelMe_OursIndividual3_test_std.csv
LabelMe_OursIndividual3_trainAnn.csv
LabelMe_OursIndividual3_trainAnn_std.csv
LabelMe_OursIndividual3_train.csv
LabelMe_OursIndividual3_train_std.csv
LabelMe_softMV_test.csv
LabelMe_softMV_test_std.csv
LabelMe_softMV_train.csv
LabelMe_softMV_train_std.csv
LabelMe_UpperBound_test.csv
LabelMe_UpperBound_train.csv
results_ind.txt


['softMV', 'OursIndividual2', 'hardMV', 'OursIndividual3']

In [22]:
t_acc = compare_table("acc")
t_acc_std = compare_table("acc", std=True)

t_norm_R = compare_table("NormF","R")
t_norm_R_std = compare_table("NormF","R", std=True)
t_js_R = compare_table("JS","R")
t_js_R_std = compare_table("JS","R",  std=True)

t_norm_G = compare_table("NormF","G")
t_norm_G_std = compare_table("NormF","G", std=True)
t_js_G = compare_table("JS","G")
t_js_G_std = compare_table("JS","G", std=True)

t_epochs = compare_table("epochs")

In [23]:
t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True)

,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),0.769,-,0.769,-
1,softMV,0.8325,0.8155,0.0089,0.0135
2,OursIndividual2,0.8499,0.8324,0.0154,0.0191
3,hardMV,0.8021,0.8005,0.0089,0.0125
4,OursIndividual3,0.8456,0.8366,0.0105,0.0138


In [24]:
from IPython.display import display
display(t_js_R.merge(t_js_R_std.iloc[:,1:], left_index=True, right_index=True))
display(t_norm_R.merge(t_norm_R_std.iloc[:,1:], left_index=True, right_index=True))

,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,softMV,0.2163,-,0,-
2,OursIndividual2,0.2018,-,0.0023,-
3,hardMV,0.2252,-,0,-
4,OursIndividual3,0.2,-,0.0049,-


,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,softMV,0.172,-,0,-
2,OursIndividual2,0.1595,-,0.0012,-
3,hardMV,0.1812,-,0,-
4,OursIndividual3,0.1626,-,0.0035,-


In [26]:
display(t_js_G.merge(t_js_G_std.iloc[:,1:], left_index=True, right_index=True))
display(t_norm_G.merge(t_norm_G_std.iloc[:,1:], left_index=True, right_index=True))

,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,softMV,0.0242,-,0,-
2,OursIndividual2,0.0512,-,0.0032,-
3,hardMV,0.0347,-,0,-
4,OursIndividual3,0.0517,-,0.004,-


,Methods,Train_x,Test_x,Train_y,Test_y
0,Ann(tr),-,-,-,-
1,softMV,0.0566,-,0,-
2,OursIndividual2,0.0545,-,0.0026,-
3,hardMV,0.0736,-,0,-
4,OursIndividual3,0.0674,-,0.0036,-


## T change

In [17]:
folder = "./I_fix/simple_T/"

In [18]:
to_check = [100,500,1500,3500,6000,10000]

if 'simple' in folder.lower():
    T_data = 5 #for now is this..
    N = 2500
elif "cifar" in folder.lower():
    T_data = 3
    N = 50000
D_t = np.asarray([N*T_data/T for T in to_check])

files_f = [f for f in os.listdir(folder) if ".pickle" in f]
dic_res = {}
for file in files_f:
    #if len(file.split("_")) >2:
    _, method_name, method_type = (file.split(".pickle")[0]).split("_")
    with open(folder+file, 'rb') as handle:
        if method_name in dic_res:
            dic_res[method_name][method_type] = pickle.load(handle)
        else:
            dic_res[method_name] = {}
            dic_res[method_name][method_type] = pickle.load(handle)
methods_list = list(dic_res.keys())
methods_list

['softMV', 'OursIndividual2', 'OursIndividual3', 'hardMV']

In [19]:
t, t_std = create_df(methods_list, to_check,"Accuracy", file_re=folder+'results_ind.txt')
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

,100,500_x,1500_x,3500_x,6000_x,10000_x,500_y,1500_y,3500_y,6000_y,10000_y
Ann(tr),0.7188,0.7155,0.7151,0.7281,0.7049,0.7350,0.0000,0.0000,0.0000,0.0000,0.0000
softMV,0.6856,0.6835,0.6708,0.6792,0.6545,0.6631,0.0459,0.0351,0.0374,0.0133,0.0288
OursIndividual2,0.8801,0.8994,0.8970,0.9401,0.9451,0.9018,0.1477,0.1473,0.0085,0.0049,0.0170
OursIndividual3,0.8993,0.8850,0.8512,0.7590,0.6937,0.6889,0.1756,0.1870,0.2070,0.2109,0.1377
hardMV,0.8152,0.8487,0.7892,0.7842,0.7505,0.7751,0.0300,0.0527,0.0353,0.0584,0.0429


In [20]:
t, t_std = create_df(methods_list,to_check, "(R) JS mean")
#t, t_std = create_df(methods_list,to_check, "(R) NormF mean")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

,100,500_x,1500_x,3500_x,6000_x,10000_x,500_y,1500_y,3500_y,6000_y,10000_y
softMV,0.0920,0.0896,0.0897,0.0868,0.0870,0.0864,0.0000,0.0000,0.0000,0.0000,0.0000
OursIndividual2,0.0327,0.0323,0.0727,0.1258,0.1671,0.2418,0.0346,0.0457,0.0030,0.0017,0.0039
OursIndividual3,0.0280,0.0389,0.0761,0.1289,0.1789,0.2206,0.0609,0.0560,0.0586,0.0451,0.0227
hardMV,0.0916,0.0877,0.0903,0.0881,0.0871,0.0853,0.0000,0.0000,0.0000,0.0000,0.0000


In [6]:
t, t_std = create_df(methods_list,to_check,"(G) JS")
#t, t_std = create_df(methods_list,to_check,"(G) NormF")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

,100,500_x,1500_x,3500_x,6000_x,10000_x,500_y,1500_y,3500_y,6000_y,10000_y
softMV,0.0143,0.0135,0.0140,0.0140,0.0140,0.0153,0.0000,0.0000,0.0000,0.0000,0.0000
OursIndividual2,0.0190,0.0140,0.0162,0.0119,0.0122,0.0256,0.0255,0.0362,0.0030,0.0031,0.0011
OursIndividual3,0.0149,0.0176,0.0229,0.0337,0.0431,0.0547,0.0418,0.0462,0.0544,0.0449,0.0209
hardMV,0.0173,0.0153,0.0170,0.0176,0.0170,0.0160,0.0000,0.0000,0.0000,0.0000,0.0000


In [ ]:
print(tabulate(t,headers='keys',tablefmt='latex'))

In [40]:
t, _ = create_df(methods_list,to_check,"Iner JS")
#t, _ = create_df(methods_list,to_check,"Iner NormF")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

,100,500,1500,3500,6000,10000
Ann(tr),0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
DS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
OursIndividual3,0.2886,0.3125,0.3407,0.4686,0.5989,0.7124
hardMV,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
OursIndividual2,0.2879,0.2891,0.2336,0.3825,0.5453,0.6709
softMV,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
OursGlobal,0.2297,0.2429,0.2102,0.2404,0.2049,0.1949
Raykar,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
